# The Puzzle 

Three players A, B, C play the following game. First, A picks a real number between $0$ and $1$ (both inclusive), then B picks a number in the same range (different from A’s choice) and finally C picks a number, also in the same range, (different from the two chosen numbers). We then pick a number in the range uniformly randomly. Whoever’s number is closest to this random number wins the game. Assume that A, B and C all play optimally and their sole goal is to maximise their chances of winning. Also assume that if one of them has several optimal choices, then that player will randomly pick one of the optimal choices.

- If A chooses 0, then what is the best choice for B?
- What is the best choice for A?
- Can you write a program to figure out the best choice for the first player when the game is played among four players?

<i style="font-size:8pt">As of July 2019, this puzzle is to be solved in order to apply for the Quantitative Researcher position at Optiver.</i>

# 1) Best Strategy for B, given that A chose $a=0$.

Given that $a=0$ and a choice of B, $b$, the win probability for the choice $c$ of C is:
- $1-b$ if $b<c$,
- $b/2$ if $b>c$;

where we have assumed that, if C chooses a number bigger than $b$, he will choose $b+\epsilon$ (with $\epsilon$ aribitrarily small).

By solving $1-b=b/2$ we find the best strategy for B.

In fact, if B chooses <b>b=<i>2/3</i></b>, it follows that C has no incentives to choose $c=b+\epsilon$, or any number between $0$ and $b$.

In particular, if B chooses $b<2/3$, C will gain by choosing $c=b+\epsilon$, and if B chooses $b>2/3$, C will gain by choosing $\forall c\in(0,b)$.

# 2) Best strategy for A with three players.

Let's assume the choice of A to be $a<0.5$ without loss of generality (the case with $a>0.5$ would be symmetrycal).

For continuity with the previous question, if $a=\epsilon$, B would still choose $b=\frac{2}{3}(1-\epsilon)+\epsilon$. In this case A would have the same chances to win as in the previous exercise, plus $\epsilon$.

And this would still be true for some time as $a$ departs from $0$.

Clearly, however, this can not be true indefinitely because if $a=0.5$, B will choose $b=0.5\pm \epsilon$ and $c=0.5\mp\epsilon$, always leaving A with no chance to win. Somewhere in between $0$ and $0.5$ there must be an optimum.

If B chooses $b=\frac{2}{3}(1-a)+a=\frac{2}{3}+\frac{1}{3}a$ he has probability to win of at least $(1-a)/3$. But if $a$ is big enough, $b$ will be better off choosing $b=a-\epsilon$, getting a probability of victory of $a$.

The optimum value for the choice of A is reached when the probability of B to win for the two possibly choices ($b=a-\epsilon$ and $b=\frac{2}{3}+\frac{1}{3}a$) are the same, i.e. when $(1-a)/3=a$, i.e. when $a=1/4$.

By symmetry, the other optimal value for A is $a=3/4$.

Below, a simulation of this problem for 3 and 4 players.

# 2b) Best strategy for A (and B and C) with 3 players: simulation

In [1]:
import numpy as np
import anytree as tree
import gc

For the sake of computation, we discretize the problem, limiting the possible choices of the players to: $0$, $1/N$, $2/N$, ..., $(N-1)/N$.

In [2]:
N = 100 # number of possible choices

In [3]:
def compute_points(choice_set,N=24):
    n = len(choice_set)
    order = np.argsort(choice_set)
    backorder = np.argsort(order)
    ochoices = np.array([choice_set[i] for i in order])
    #print(order,ochoices)
    diff = [(ochoices[i+1]-ochoices[i])/2 for i in range(n-1)]
    #print(diff)
    outcomes = [ochoices[0]+diff[0]]+[diff[i]+diff[i+1] for i in range(n-2)]+[N-1-ochoices[-1]+diff[-1]]
    #print(outcomes)
    return [outcomes[i] for i in backorder]

We build a tree with all the possible outcomes

In [4]:
root = tree.AnyNode(children=[
    tree.AnyNode(a=i,children=[
        tree.AnyNode(a=i,b=j,children=[
            tree.AnyNode(a=i,b=j,c=k,outcome=compute_points([i,j,k],N=N)) for k in range(N) if k not in [i,j]
        ]) for j in range(N) if j!=i
    ]) for i in range(N)
])

Write a function "filter_tree" that at a specific level of the tree is going to keep only the best choice for the player at that level. It gets rid of all the others leaves which give worst outcomes.

In [5]:
def filter_tree(upper,outcome_pref,final=False):
    leaves = list(upper.children)
    vals = [leaf.outcome[outcome_pref] for leaf in leaves]
    i_max = np.random.choice(np.argwhere(vals == np.amax(vals)).flatten())
    #i_max = np.argmax(vals)
    upper.children = [leaves[i_max]]
    upper.outcome = leaves[i_max].outcome
    if final == True:
        return leaves[i_max].descendants[-1]

Starting from the last player, we choose his best strategy, given the choices of the previous players. We then move to the second-to-last player and do the same thing. We keep on filtering the tree up to when there is only one path left.

In [6]:
for a in range(N):
    for b in range(N-1):
        filter_tree(root.children[a].children[b],-1)
for a in range(N):
    filter_tree(root.children[a],-2)
finalnode = filter_tree(root,-3,final=True)
_=gc.collect() # collect the garbage

In [7]:
odds = finalnode.outcome
print(f"The choices will be:\na = {finalnode.a/N} with winning probability = {odds[0]/N}\n\
b = {finalnode.b/N} with winning probability = {odds[1]/N}\n\
c = {finalnode.c/N} with winning probability = {odds[2]/N}")

The choices will be:
a = 0.74 with winning probability = 0.3
b = 0.24 with winning probability = 0.44
c = 0.64 with winning probability = 0.25


Obviously in the result above there are rounding errors and there is a degree of randomness. When a player has many optimal options, given the choices of the previous players, we chose the player's strategy at random, among the optimal choices.

# Same as above, but for 4 players

The case with 4 players works in the same way. However, the tree will be deeper, and we will need an extra turn of filtering to reach the optimal solution.

In [8]:
N = 40 # number of possible choices 

In [9]:
root = tree.AnyNode(children=[
    tree.AnyNode(a=a,children=[
        tree.AnyNode(a=a,b=b,children=[
            tree.AnyNode(a=a,b=b,c=c,children=[
                tree.AnyNode(a=a,b=b,c=c,d=d,outcome=compute_points([a,b,c,d],N=N)) for d in range(N) if d not in [a,b,c] 
            ]) for c in range(N) if c not in [a,b]
        ]) for b in range(N) if b!=a
    ]) for a in range(N)
])

In [10]:
for a in range(N):
    for b in range(N-1):
        for c in range(N-2):
            filter_tree(root.children[a].children[b].children[c],-1)

for a in range(N):
    for b in range(N-1):
        filter_tree(root.children[a].children[b],-2)

for a in range(N):
    filter_tree(root.children[a],-3)
finalnode = filter_tree(root,-4,final=True)
_=gc.collect() # collect the garbage

In [11]:
odds = finalnode.outcome
print(f"The choices will be:\na = {finalnode.a/N} with winning probability = {odds[0]/N}\n\
b = {finalnode.b/N} with winning probability = {odds[1]/N}\n\
c = {finalnode.c/N} with winning probability = {odds[2]/N}\n\
d = {finalnode.d/N} with winning probability = {odds[3]/N}")

The choices will be:
a = 0.175 with winning probability = 0.1875
b = 0.525 with winning probability = 0.3
c = 0.8 with winning probability = 0.3125
d = 0.2 with winning probability = 0.175


The best choice for A is $a = 1/5 - \epsilon$